In [19]:
!tree -L 3


.
├── README.md
├── data
│   ├── 5b1802ef89579c0ed05e707f96177cab.grib
│   ├── 5b1802ef89579c0ed05e707f96177cab.grib.5b7b6.idx
│   ├── DEM.tif
│   ├── LCFM_LCM-10_V100_2020_N21E084_cog
│   │   └── LCFM_LCM-10_V100_2020_N21E084_MAP.tif
│   ├── aspect.tif
│   ├── features_full_stack.tif
│   ├── features_static_stack.tif
│   ├── features_weather_stack.tif
│   ├── fire_vriis.csv
│   ├── fuel_reclass.tif
│   ├── slope.tif
│   └── tiles
│       ├── tile_0_0.tif
│       ├── tile_0_1024.tif
│       ├── tile_0_1280.tif
│       ├── tile_0_1536.tif
│       ├── tile_0_1792.tif
│       ├── tile_0_2048.tif
│       ├── tile_0_2304.tif
│       ├── tile_0_256.tif
│       ├── tile_0_2560.tif
│       ├── tile_0_2816.tif
│       ├── tile_0_3072.tif
│       ├── tile_0_3328.tif
│       ├── tile_0_512.tif
│       ├── tile_0_768.tif
│       ├── tile_1024_0.tif
│       ├── tile_1024_1024.tif
│       ├── tile_1024_1280.tif
│       ├── tile_1024_1536.tif
│       ├── tile_1024_1792.tif
│       ├── tile_1024_2048.

# STACKING

In [12]:
import numpy as np
import rasterio
from rasterio.enums import Resampling

# Data Paths
dem_fp        = "data/DEM.tif"
lulc_fp       = "data/LCFM_LCM-10_V100_2020_N21E084_cog/LCFM_LCM-10_V100_2020_N21E084_MAP.tif"
slope_fp      = "data/slope.tif"
aspect_fp     = "data/aspect.tif"
fuel_fp       = "data/fuel_reclass.tif"

# READ DEM
with rasterio.open(dem_fp) as src:
    dem      = src.read(1).astype("float32")
    transform= src.transform
    profile  = src.profile.copy()

# compute pixel size
xres = transform.a
yres = -transform.e

# gradients
dz_dy, dz_dx = np.gradient(dem, yres, xres)

# slope in degrees
slope = np.degrees(np.arctan(np.sqrt(dz_dx**2 + dz_dy**2)))

# aspect in degrees
aspect = np.degrees(np.arctan2(dz_dy, -dz_dx))
aspect = np.where(aspect < 0,
                  90.0 - aspect,
                  360.0 - aspect + 90.0)
aspect = np.where((dz_dx == 0) & (dz_dy == 0), -1, aspect)

# update profile for single band float outputs
profile.update(
    dtype=rasterio.float32,
    count=1,
    compress="lzw",
    nodata=-1
)

# slope
with rasterio.open(slope_fp, "w", **profile) as dst:
    dst.write(slope.astype(rasterio.float32), 1)

# aspect
with rasterio.open(aspect_fp, "w", **profile) as dst:
    dst.write(aspect.astype(rasterio.float32), 1)

print(" Slope & aspect written to:", slope_fp, aspect_fp)


# READ LULC & reclassify to fuel score
#    WorldCover codes → fuel: 10->3, 20->2, 30->1, 40->1, others (50+) ->0
with rasterio.open(lulc_fp) as src:
    lulc = src.read(1)
    fuel_profile = src.profile.copy()
    
# build empty fuel array
fuel = np.zeros_like(lulc, dtype=np.uint8)

# apply mapping
fuel[np.isin(lulc, [10])] = 3
fuel[np.isin(lulc, [20])] = 2
fuel[np.isin(lulc, [30, 40])] = 1
# all other values remain 0

# update profile
fuel_profile.update(
    dtype=rasterio.uint8,
    count=1,
    compress="lzw",
    nodata=0
)

# write fuel raster
with rasterio.open(fuel_fp, "w", **fuel_profile) as dst:
    dst.write(fuel, 1)

print("Fuel raster written to:", fuel_fp)


 Slope & aspect written to: data/slope.tif data/aspect.tif
Fuel raster written to: data/fuel_reclass.tif


In [13]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
from rasterio.warp import reproject

# Input rasters
inputs = {
    "slope" : "data/slope.tif",
    "aspect": "data/aspect.tif",
    "fuel"  : "data/fuel_reclass.tif"
}
output_path = "data/features_static_stack.tif"

# Open reference (slope) to get grid/CRS
with rasterio.open(inputs["slope"]) as ref:
    ref_meta = ref.meta.copy()
    ref_transform, ref_crs = ref.transform, ref.crs
    height, width = ref.height, ref.width

# Prepare output metadata
ref_meta.update({
    "count": len(inputs),
    "dtype": "float32",
    "compress": "lzw",
    "nodata": -9999
})

# Create stacked file
with rasterio.open(output_path, "w", **ref_meta) as dst:
    for idx, (name, path) in enumerate(inputs.items(), start=1):
        with rasterio.open(path) as src:
            src_data = src.read(1)
            # If same grid, no reprojection needed
            if (src.transform == ref_transform 
                and src.crs == ref_crs 
                and src.width == width 
                and src.height == height):
                band = src_data.astype("float32")
            else:
                # Allocate empty array for resampled data
                band = np.empty((height, width), dtype="float32")
                # Reproject/resample into our reference grid
                reproject(
                    source=src_data,
                    destination=band,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=ref_transform,
                    dst_crs=ref_crs,
                    resampling=Resampling.bilinear,
                )
        dst.write(band, idx)
        dst.set_band_description(idx, name)

print(f" Static feature stack saved to {output_path}")


 Static feature stack saved to data/features_static_stack.tif


In [14]:
!pip install xarray cfgrib rasterio

In [15]:
import xarray as xr

grib_fp = "data/5b1802ef89579c0ed05e707f96177cab.grib"
ds = xr.open_dataset(grib_fp, engine="cfgrib")

# List all data variables in the dataset
print(ds)
print("\nData variables available:")
for name in ds.data_vars:
    print(" -", name)


skipping variable: paramId==228 shortName='tp'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/opt/anaconda3/lib/python3.12/site-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1612137600, 1612141200, 1612144800, ..., 1617224400, 1617228000,
       1617231600])) new_value=Variable(dimensions=('time',), data=array([1612116000, 1612159200, 1612202400, 1612245600, 1612288800,
       1612332000, 1612375200, 1612418400, 1612461600, 1612504800,
       1612548000, 1612591200, 1612634400, 1612677600, 1612720800,
       1612764000, 1612807200, 1612850400, 1612893600, 1612936800,
       1612980000, 1613023200, 1613066400, 1613109600, 1613152800,
       1613196000, 1613239200, 1613282400, 1

<xarray.Dataset> Size: 136kB
Dimensions:     (time: 1416, latitude: 2, longitude: 2)
Coordinates:
    number      int64 8B ...
  * time        (time) datetime64[ns] 11kB 2021-02-01 ... 2021-03-31T23:00:00
    step        timedelta64[ns] 8B ...
    surface     float64 8B ...
  * latitude    (latitude) float64 16B 21.93 21.68
  * longitude   (longitude) float64 16B 86.0 86.25
    valid_time  (time) datetime64[ns] 11kB ...
Data variables:
    u10         (time, latitude, longitude) float32 23kB ...
    v10         (time, latitude, longitude) float32 23kB ...
    d2m         (time, latitude, longitude) float32 23kB ...
    t2m         (time, latitude, longitude) float32 23kB ...
    sp          (time, latitude, longitude) float32 23kB ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             Eu

/opt/anaconda3/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


# TILING

In [16]:
import xarray as xr
import numpy as np
import rasterio
from rasterio.warp import reproject, Resampling
from rasterio.transform import from_bounds

# Paths
grib_fp      = "data/5b1802ef89579c0ed05e707f96177cab.grib"
ref_fp       = "data/features_static_stack.tif"
weather_fp   = "data/features_weather_stack.tif"

vars_info = {
    "t2m": "mean",
    "d2m": "mean",
    "u10": "mean",
    "v10": "mean",
    "sp":  "mean"
}

# Open ref for grid
with rasterio.open(ref_fp) as ref:
    meta   = ref.meta.copy()
    transform, crs = ref.transform, ref.crs
    h, w  = ref.height, ref.width

# update for 5 weather bands
meta.update(count=len(vars_info), dtype="float32", compress="lzw", nodata=-9999)

# Load GRIB via xarray
ds = xr.open_dataset(grib_fp, engine="cfgrib")
lons, lats = ds.longitude.values, ds.latitude.values
src_transform = from_bounds(lons.min(), lats.min(), lons.max(), lats.max(),
                            lons.size, lats.size)
src_crs = "EPSG:4326"

# Write weather stack
with rasterio.open(weather_fp, "w", **meta) as dst:
    for i, (var, agg) in enumerate(vars_info.items(), start=1):
        arr = getattr(ds[var], agg)(dim="time").values.squeeze().astype("float32")
        buf = np.empty((h, w), dtype="float32")
        reproject(arr, buf,
                  src_transform=src_transform, src_crs=src_crs,
                  dst_transform=transform,  dst_crs=crs,
                  resampling=Resampling.bilinear)
        dst.write(buf, i)
        dst.set_band_description(i, var)

print(" Weather stack written to", weather_fp)


skipping variable: paramId==228 shortName='tp'
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/cfgrib/dataset.py", line 725, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/opt/anaconda3/lib/python3.12/site-packages/cfgrib/dataset.py", line 641, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1612137600, 1612141200, 1612144800, ..., 1617224400, 1617228000,
       1617231600])) new_value=Variable(dimensions=('time',), data=array([1612116000, 1612159200, 1612202400, 1612245600, 1612288800,
       1612332000, 1612375200, 1612418400, 1612461600, 1612504800,
       1612548000, 1612591200, 1612634400, 1612677600, 1612720800,
       1612764000, 1612807200, 1612850400, 1612893600, 1612936800,
       1612980000, 1613023200, 1613066400, 1613109600, 1613152800,
       1613196000, 1613239200, 1613282400, 1

 Weather stack written to data/features_weather_stack.tif


In [17]:
import rasterio
from rasterio.merge import merge

static_fp  = "data/features_static_stack.tif"
weather_fp = "data/features_weather_stack.tif"
out_fp     = "data/features_full_stack.tif"

# open both
srcs = [rasterio.open(static_fp), rasterio.open(weather_fp)]

# same grid/crs: just merge bands
meta = srcs[0].meta.copy()
meta.update(count=srcs[0].count + srcs[1].count)

with rasterio.open(out_fp, "w", **meta) as dst:
    # write static bands
    for b in range(1, srcs[0].count+1):
        dst.write(srcs[0].read(b), b)
    # write weather bands
    for b in range(1, srcs[1].count+1):
        dst.write(srcs[1].read(b), srcs[0].count + b)

print("Full feature stack saved to", out_fp)


Full feature stack saved to data/features_full_stack.tif


In [18]:
import os
import rasterio
from rasterio.windows import Window
import numpy as np

stack_fp = "data/features_full_stack.tif"
out_dir  = "data/tiles"
tile_size = 256
os.makedirs(out_dir, exist_ok=True)

with rasterio.open(stack_fp) as src:
    for i in range(0, src.height, tile_size):
        for j in range(0, src.width, tile_size):
            if i + tile_size <= src.height and j + tile_size <= src.width:
                window = Window(j, i, tile_size, tile_size)
                data = src.read(window=window)
                out_meta = src.meta.copy()
                out_meta.update({
                    "height": tile_size,
                    "width": tile_size,
                    "transform": rasterio.windows.transform(window, src.transform)
                })
                tile_fp = os.path.join(out_dir, f"tile_{i}_{j}.tif")
                with rasterio.open(tile_fp, "w", **out_meta) as dst:
                    dst.write(data)
